In [1]:
import os 

In [5]:
os.getcwd()

'C:\\Users\\HP\\NFT assignment 2'

In [6]:
import glob
import numpy as np
import pandas as pd 


In [14]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [17]:
def prepare_dataset(path,label):
    x_train=[]
    y_train=[]
    all_images_path=glob.glob(path+'/*.jpg')
    for img_path in all_images_path :
            img=load_img(img_path, target_size=(150,150))
            img=img_to_array(img)
            img=img/255.0
            x_train.append(img)
            y_train.append(label)
    return np.array(x_train),np.array(y_train)

In [19]:
def prepare_dataset_train(path,label):
    x_train=[]
    y_train=[]
    inp  = 0 
    all_images_path=glob.glob(path+'/*.jpg')
    for img_path in all_images_path :
            img=load_img(img_path, target_size=(150,150))
            img=img_to_array(img)
            img=img/255.0
            if(inp%4 == 0) :
                x_train.append(img)
                y_train.append(label)
            inp += 1 
    return np.array(x_train),np.array(y_train)

In [8]:
paths=glob.glob('E://seg_train/seg_train/*')
l=len('E://seg_train/seg_train/')
labels=[]
for path in paths:
    labels.append(path[l:])
    print(labels)

['buildings']
['buildings', 'forest']
['buildings', 'forest', 'glacier']
['buildings', 'forest', 'glacier', 'mountain']
['buildings', 'forest', 'glacier', 'mountain', 'sea']
['buildings', 'forest', 'glacier', 'mountain', 'sea', 'street']


In [15]:
testX_building, testY_building  = prepare_dataset("E://seg_test/seg_test/buildings/",0)
testX_forest,testY_forest  = prepare_dataset("E://seg_test/seg_test/forest/",1)
testX_glacier,testY_glacier  = prepare_dataset("E://seg_test/seg_test/glacier/",2)
testX_mount,testY_mount  = prepare_dataset("E://seg_test/seg_test/mountain/",3)
testX_sea,testY_sea  = prepare_dataset("E://seg_test/seg_test/sea/",4)
testX_street,testY_street  = prepare_dataset("E://seg_test/seg_test/street/",5)

x_test=np.concatenate((testX_building,testX_forest,testX_glacier,testX_mount,testX_sea,testX_street),axis=0)
y_test=np.concatenate((testY_building,testY_forest,testY_glacier,testY_mount,testY_sea,testY_street),axis=0)

In [16]:
x_test.shape 

(3000, 150, 150, 3)

In [20]:
trainX_building, trainY_building  = prepare_dataset("E://seg_train/seg_train/buildings/",0)
trainX_forest,trainY_forest  = prepare_dataset("E://seg_train/seg_train/forest/",1)
trainX_glacier,trainY_glacier  = prepare_dataset("E://seg_train/seg_train/glacier/",2)
trainX_mount,trainY_mount  = prepare_dataset("E://seg_train/seg_train/mountain/",3)
trainX_sea,trainY_sea  = prepare_dataset("E://seg_train/seg_train/sea/",4)
trainX_street,trainY_street  = prepare_dataset("E://seg_train/seg_train/street/",5)

In [21]:
x_train =np.concatenate((trainX_building,trainX_forest,trainX_glacier,trainX_mount,trainX_sea,trainX_street),axis=0)
y_train = np.concatenate((trainY_building,trainY_forest,trainY_glacier,trainY_mount,trainY_sea,trainY_street),axis=0)

In [22]:
import os
from tensorflow.keras import layers
from tensorflow.keras import Model
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.optimizers import RMSprop

In [23]:
pre_trained_model = InceptionV3(input_shape = (150, 150, 3), 
                                include_top = False, 
                                weights = "imagenet")

In [24]:
for layer in pre_trained_model.layers:
     layer.trainable = False

In [25]:
last_layer = pre_trained_model.get_layer('mixed7')
print('last layer output shape: ', last_layer.output_shape)
last_output = last_layer.output

last layer output shape:  (None, 7, 7, 768)


In [26]:
x = layers.Flatten()(last_output)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.2)(x)                  
x = layers.Dense(6, activation='softmax')(x)  

In [27]:
model = Model(pre_trained_model.input, x) 

In [28]:
model.compile(optimizer = RMSprop(lr=0.0001), 
              loss = 'sparse_categorical_crossentropy', 
              metrics = ['acc'])

In [29]:
history=model.fit(x_train,y_train,epochs=1,validation_data=(x_test,y_test))

439/439 [==============================] - 574s 1s/step - loss: 0.4157 - acc: 0.8641 - val_loss: 0.3288 - val_acc: 0.8817
